# Imports

In [95]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [96]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 24                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [97]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [98]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [99]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [100]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs),
                              ("day", day_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [101]:
dataset = Dataset(dirpath=os.path.join("data_24h", "depresjon"))
condition = dataset.condition
control = dataset.control

In [102]:
condition[0]

,timestamp,date,activity
0,2003-06-20 10:30:00,2003-06-20,25
1,2003-06-20 10:31:00,2003-06-20,36
2,2003-06-20 10:32:00,2003-06-20,83
3,2003-06-20 10:33:00,2003-06-20,19
4,2003-06-20 10:34:00,2003-06-20,9
...,...,...,...
1435,2003-06-21 10:25:00,2003-06-21,12
1436,2003-06-21 10:26:00,2003-06-21,32
1437,2003-06-21 10:27:00,2003-06-21,265
1438,2003-06-21 10:28:00,2003-06-21,155


In [103]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [104]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [105]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [106]:
datasets["full_24h"].dropna(inplace=True)
datasets["full_24h"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,1.550000,691.683350,141.303329,94.250000,25246.462891,4.017333,1.999729,1.101751,140.183331,115.535706,...,7.056636e+05,4.037811e+05,3.285897e+11,-0.644289,0.955567,0.812324,7.698562e+05,6.469822e+05,3.269354,0.731094
1,0.000000,166.016663,77.510666,83.750000,2515.122803,-0.994866,0.062486,0.633948,68.666672,76.574600,...,7.105572e+04,7.937672e+04,6.719426e+08,-0.556905,-0.554086,0.364810,3.766367e+04,7.262927e+04,3.588991,0.903982
2,0.000000,1083.199951,232.162231,206.983337,52457.531250,5.615858,1.980165,0.966603,298.649994,204.477234,...,1.801848e+06,1.463198e+06,9.874039e+11,-0.358756,0.865229,0.551480,8.479128e+05,1.731729e+06,3.493078,0.859510
3,8.200000,1146.166626,383.709717,201.825012,186697.281250,-1.178148,0.701137,1.102363,825.245844,347.231659,...,3.672580e+06,1.105212e+06,1.906912e+13,-0.063627,1.227047,1.189033,3.419727e+06,3.069298e+06,2.823935,0.487504
4,0.616667,294.033325,56.151924,25.100000,6219.544434,2.234426,1.756296,1.376101,69.760004,41.694355,...,1.560999e+05,1.322794e+05,8.560280e+09,-1.190034,0.450418,0.592709,1.375921e+05,1.524360e+05,3.445226,0.819626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,2.316667,740.166687,312.192352,323.333313,69233.890625,-1.492479,0.096636,0.825079,499.283340,301.987488,...,1.040252e+06,1.039700e+06,2.477260e+11,-0.869785,-0.521578,0.478462,5.855452e+05,1.062303e+06,3.495905,0.808007
1059,0.750000,1828.250000,353.719452,281.850006,195576.265625,3.833997,1.983241,1.223932,408.370827,266.735016,...,6.786808e+06,4.923788e+06,1.662164e+13,-0.627133,0.822168,0.600719,4.836770e+06,6.520742e+06,3.454776,0.835591
1060,7.550000,586.833313,272.634735,356.591675,47321.957031,-1.600275,-0.098908,0.781103,436.470829,268.144989,...,7.730123e+05,5.139775e+05,3.929674e+11,-1.430280,0.471400,0.810946,1.197435e+06,7.370223e+05,3.212892,0.658163
1062,2.016667,957.150024,393.095123,398.108337,98765.867188,-1.317137,0.079247,0.782644,633.341680,382.582489,...,1.737632e+06,1.778565e+06,1.157349e+12,-1.026521,0.173076,0.619119,1.876138e+06,1.700684e+06,3.386425,0.718106


## Psykose

In [107]:
dataset = Dataset(dirpath=os.path.join("data_24h", "psykose"))
condition = dataset.condition
control = dataset.control

In [108]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [109]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [110]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
datasets["full_24h"].dropna(inplace=True)

# Classification

## Classifiers, parameters, constants

In [111]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    )
}


param_grids = {
    "LR": {
        "C": [0.5, 1, 2, 5],
        "class_weight": [None, "balanced"],
        "l1_ratio": [ 0.4, 0.45, 0.5,0.55, 0.6, 0.65]
    },
    "SVM": {
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Classification

In [112]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [113]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["full_24h"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
print(datasets['full_24h'].shape)
print(y.shape)

(991, 24)
(991,)


In [114]:
for part in ["full_24h"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: full_24h
  LR
{'accuracy': 0.628140703517588, 'balanced_accuracy': 0.5, 'f1': 0.0, 'precision': 1.0, 'recall': 0.0, 'specificity': 1.0, 'ROC_AUC': 0.5, 'MCC': 0.0}
{'accuracy': 0.6262626262626263, 'balanced_accuracy': 0.5, 'f1': 0.0, 'precision': 1.0, 'recall': 0.0, 'specificity': 1.0, 'ROC_AUC': 0.5, 'MCC': 0.0}
{'accuracy': 0.6262626262626263, 'balanced_accuracy': 0.5, 'f1': 0.0, 'precision': 1.0, 'recall': 0.0, 'specificity': 1.0, 'ROC_AUC': 0.5, 'MCC': 0.0}
{'accuracy': 0.6313131313131313, 'balanced_accuracy': 0.5, 'f1': 0.0, 'precision': 1.0, 'recall': 0.0, 'specificity': 1.0, 'ROC_AUC': 0.5, 'MCC': 0.0}
{'accuracy': 0.6313131313131313, 'balanced_accuracy': 0.5, 'f1': 0.0, 'precision': 1.0, 'recall': 0.0, 'specificity': 1.0, 'ROC_AUC': 0.5, 'MCC': 0.0}
    accuracy: 0.6287 +- 0.0023
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.0000 +- 0.0000
    precision: 1.0000 +- 0.0000
    recall: 0.0000 +- 0.0000
    specificity: 1.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    M